In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import pybedtools

In [2]:
splice_site_path= "/data/private/pdutta/Collab_data/Oliver_data/New_data/splice_sites.txt" 

In [3]:
df_splice = pd.read_csv(splice_site_path, sep="\t", names= ['chr', 'position', 'splice_type', 'RefSeq_Transcript_ID', 'strand', 'gene'])

In [4]:
df_splice

,chr,position,splice_type,RefSeq_Transcript_ID,strand,gene
0,chr1,966614,D,NM_001160184.2,+,PLEKHN1
1,chr1,966614,D,NM_001367552.1,+,PLEKHN1
2,chr1,966614,D,NM_032129.3,+,PLEKHN1
3,chr1,966703,A,NM_001160184.2,+,PLEKHN1
4,chr1,966703,A,NM_001367552.1,+,PLEKHN1
...,...,...,...,...,...,...
14265,chrX,151700505,A,NM_024082.4,+,PRRG3
14266,chrX,154478058,A,NM_006014.5,-,LAGE3
14267,chrX,154478282,D,NM_006014.5,-,LAGE3
14268,chrX,154478411,A,NM_006014.5,-,LAGE3


In [5]:
df_splice.groupby("splice_type").size()

splice_type
A    7135
D    7135
dtype: int64

In [6]:
# Define a custom aggregation function
def custom_agg(series):
    # If all values are the same, return any one of them; otherwise, return the list of unique values
    if series.nunique() == 1:
        return series.iloc[0]
    else:
        return list(series.unique())

In [7]:
unique_splice_df = df_splice.groupby(['chr', 'position', 'splice_type']).agg(custom_agg).reset_index()
unique_splice_df

,chr,position,splice_type,RefSeq_Transcript_ID,strand,gene
0,chr1,966614,D,"[NM_001160184.2, NM_001367552.1, NM_032129.3]",+,PLEKHN1
1,chr1,966703,A,"[NM_001160184.2, NM_001367552.1, NM_032129.3]",+,PLEKHN1
2,chr1,966803,D,"[NM_001160184.2, NM_001367552.1, NM_032129.3]",+,PLEKHN1
3,chr1,970276,A,"[NM_001160184.2, NM_001367552.1, NM_032129.3]",+,PLEKHN1
4,chr1,970423,D,"[NM_001160184.2, NM_001367552.1, NM_032129.3]",+,PLEKHN1
...,...,...,...,...,...,...
5820,chrX,151700505,A,"[NM_001372163.1, NM_001372165.1, NM_024082.4]",+,PRRG3
5821,chrX,154478058,A,NM_006014.5,-,LAGE3
5822,chrX,154478282,D,NM_006014.5,-,LAGE3
5823,chrX,154478411,A,NM_006014.5,-,LAGE3


In [8]:
df_splice_donor= unique_splice_df[unique_splice_df['splice_type']=="D"]
df_splice_donor

,chr,position,splice_type,RefSeq_Transcript_ID,strand,gene
0,chr1,966614,D,"[NM_001160184.2, NM_001367552.1, NM_032129.3]",+,PLEKHN1
2,chr1,966803,D,"[NM_001160184.2, NM_001367552.1, NM_032129.3]",+,PLEKHN1
4,chr1,970423,D,"[NM_001160184.2, NM_001367552.1, NM_032129.3]",+,PLEKHN1
6,chr1,970601,D,"[NM_001160184.2, NM_001367552.1, NM_032129.3]",+,PLEKHN1
8,chr1,970758,D,"[NM_001160184.2, NM_001367552.1, NM_032129.3]",+,PLEKHN1
...,...,...,...,...,...,...
5814,chrX,151695544,D,NM_001372163.1,+,PRRG3
5816,chrX,151698821,D,"[NM_001372163.1, NM_001372165.1, NM_024082.4]",+,PRRG3
5819,chrX,151700156,D,"[NM_001372163.1, NM_001372165.1, NM_024082.4]",+,PRRG3
5822,chrX,154478282,D,NM_006014.5,-,LAGE3


In [9]:
df_donor_candidate = pd.read_csv("/data/private/pdutta/Collab_data/Oliver_data/New_data/Output_data/Donor/candiadte_variants.tsv", sep="\t")
df_donor_candidate

,chr_name,Donor_start,Donor_end,Strand,Variant_start,Variant_end,Reference_Nucleotide,Alternative_Nucleotides,DBSNP_ID,Info,Transcript_id,number_of_transcripts,Reference_Genome_Sequence,Alternative_Genome_Sequence,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
0,chr1,970720,970799,+,970720,970721,C,T,rs144236796,"AC=(5,);AF=(0.0003319999959785491,);AN=16852;D...","['ENST00000379407.7', 'ENST00000379410.8']",2,CGCTCGAGGGGTCCCGAGAGCACGCCTTCCAGATCACAGGTGTTTG...,TGCTCGAGGGGTCCCGAGAGCACGCCTTCCAGATCACAGGTGTTTG...,0.908698,0.203737,-0.640597,5.281620
1,chr1,970720,970799,+,970767,970768,G,T,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18790;D...","['ENST00000379407.7', 'ENST00000379410.8']",2,CGCTCGAGGGGTCCCGAGAGCACGCCTTCCAGATCACAGGTGTTTG...,CGCTCGAGGGGTCCCGAGAGCACGCCTTCCAGATCACAGGTGTTTG...,0.908698,0.214611,-0.630715,5.186764
2,chr1,974326,974405,+,974343,974344,G,A,.,"AC=(4,);AF=(0.00026080000679939985,);AN=18888;...","['ENST00000379407.7', 'ENST00000379409.6', 'EN...",4,CCAGGGAAGGTTCGCCCGAACCCTGGCTGCCTCTGACAGGTGAGTA...,CCAGGGAAGGTTCGCCCAAACCCTGGCTGCCTCTGACAGGTGAGTA...,0.716847,0.482730,-0.167826,1.439783
3,chr1,11794689,11794768,-,11794733,11794734,G,A,rs200138092,"AC=(8,);AF=(0.00030819998937658966,);AN=18912;...","['ENST00000376583.7', 'ENST00000376585.6', 'EN...",11,AACAAAGATGTATTTGCAAGGAAGGTCTGCAGGCCCTCACCAGCGG...,AACAAAGATGTATTTGCAAGGAAGGTCTGCAGGCCCTCACCAGCAG...,0.648849,0.084613,-0.366104,4.321229
4,chr1,44804741,44804820,+,44804749,44804750,C,T,.,"AC=(1,);AF=(7.111999730113894e-05,);AN=18904;D...","['ENST00000372201.5', 'ENST00000465443.5']",2,CGGTACTTCGCCTCCTACATGGAGCAGCACCTCATGAAGGTGTGAG...,CGGTACTTTGCCTCCTACATGGAGCAGCACCTCATGAAGGTGTGAG...,0.732976,0.410285,-0.236525,1.980187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,chr9,125189604,125189683,-,125189653,125189654,G,T,.,"AC=(1,);AF=(4.744000034406781e-05,);AN=18652;D...","['ENST00000373547.9', 'ENST00000415905.5', 'EN...",3,CCCCACAGCCGGAAGGGGCGAGCCCGCAAATAGGGCTCACCTTCAG...,CCCCACAGCCGGAAGGGGCGAGCCCGCAAATAGGGCTCACCTTCAG...,0.565911,0.437478,-0.072682,0.745286
115,chr9,125189604,125189683,-,125189678,125189679,C,T,.,"AC=(1,);AF=(4.742999954032712e-05,);AN=18716;D...","['ENST00000373547.9', 'ENST00000415905.5', 'EN...",3,CCCCACAGCCGGAAGGGGCGAGCCCGCAAATAGGGCTCACCTTCAG...,CCCCACAGCCGGAAGGGGCGAGCCCGCAAATAGGGCTCACCTTCAG...,0.565911,0.467558,-0.055659,0.570064
116,chr9,133345150,133345229,-,133345181,133345182,G,A,rs199628533,"AC=(1,);AF=(2.370999936829321e-05,);AN=18684;D...",['ENST00000371999.5'],1,GCCCTCCACCCTGGCCACTCACGATGTTGGCGGCTCGCACATGCAT...,GCCCTCCACCCTGGCCACTCACGATGTTGGCAGCTCGCACATGCAT...,0.531890,0.372941,-0.084544,0.933939
117,chr9,133345150,133345229,-,133345186,133345187,G,A,.,"AC=(1,);AF=(7.113000174285844e-05,);AN=18642;D...",['ENST00000371999.5'],1,GCCCTCCACCCTGGCCACTCACGATGTTGGCGGCTCGCACATGCAT...,GCCCTCCACCCTGGCCACTCACGATGTTGGCGGCTCACACATGCAT...,0.531890,0.313002,-0.116425,1.318417


In [10]:
# Find rows of df_splice_donor where the position resides between Donor_start and Donor_end of df_donor_candidate
merged_df_donor = pd.merge(df_splice_donor, df_donor_candidate, left_on="chr", right_on="chr_name")
result = merged_df_donor[(merged_df_donor['position'] >= merged_df_donor['Donor_start']) & (merged_df_donor['position'] <= merged_df_donor['Donor_end'])]
result

,chr,position,splice_type,RefSeq_Transcript_ID,strand,gene,chr_name,Donor_start,Donor_end,Strand,Variant_start,Variant_end,Reference_Nucleotide,Alternative_Nucleotides,DBSNP_ID,Info,Transcript_id,number_of_transcripts,Reference_Genome_Sequence,Alternative_Genome_Sequence,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
96,chr1,970758,D,"[NM_001160184.2, NM_001367552.1, NM_032129.3]",+,PLEKHN1,chr1,970720,970799,+,970720,970721,C,T,rs144236796,"AC=(5,);AF=(0.0003319999959785491,);AN=16852;D...","['ENST00000379407.7', 'ENST00000379410.8']",2,CGCTCGAGGGGTCCCGAGAGCACGCCTTCCAGATCACAGGTGTTTG...,TGCTCGAGGGGTCCCGAGAGCACGCCTTCCAGATCACAGGTGTTTG...,0.908698,0.203737,-0.640597,5.281620
97,chr1,970758,D,"[NM_001160184.2, NM_001367552.1, NM_032129.3]",+,PLEKHN1,chr1,970720,970799,+,970767,970768,G,T,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18790;D...","['ENST00000379407.7', 'ENST00000379410.8']",2,CGCTCGAGGGGTCCCGAGAGCACGCCTTCCAGATCACAGGTGTTTG...,CGCTCGAGGGGTCCCGAGAGCACGCCTTCCAGATCACAGGTGTTTG...,0.908698,0.214611,-0.630715,5.186764
338,chr1,974364,D,"[NM_001160184.2, NM_001367552.1, NM_032129.3]",+,PLEKHN1,chr1,974326,974405,+,974343,974344,G,A,.,"AC=(4,);AF=(0.00026080000679939985,);AN=18888;...","['ENST00000379407.7', 'ENST00000379409.6', 'EN...",4,CCAGGGAAGGTTCGCCCGAACCCTGGCTGCCTCTGACAGGTGAGTA...,CCAGGGAAGGTTCGCCCAAACCCTGGCTGCCTCTGACAGGTGAGTA...,0.716847,0.482730,-0.167826,1.439783
1203,chr1,11794728,D,"[NM_001330358.2, NM_005957.5]",-,MTHFR,chr1,11794689,11794768,-,11794733,11794734,G,A,rs200138092,"AC=(8,);AF=(0.00030819998937658966,);AN=18912;...","['ENST00000376583.7', 'ENST00000376585.6', 'EN...",11,AACAAAGATGTATTTGCAAGGAAGGTCTGCAGGCCCTCACCAGCGG...,AACAAAGATGTATTTGCAAGGAAGGTCTGCAGGCCCTCACCAGCAG...,0.648849,0.084613,-0.366104,4.321229
2956,chr1,44804779,D,NM_004073.4,+,PLK3,chr1,44804741,44804820,+,44804749,44804750,C,T,.,"AC=(1,);AF=(7.111999730113894e-05,);AN=18904;D...","['ENST00000372201.5', 'ENST00000465443.5']",2,CGGTACTTCGCCTCCTACATGGAGCAGCACCTCATGAAGGTGTGAG...,CGGTACTTTGCCTCCTACATGGAGCAGCACCTCATGAAGGTGTGAG...,0.732976,0.410285,-0.236525,1.980187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24361,chr9,125189643,D,"[NM_001123369.2, NM_002721.5]",-,PPP6C,chr9,125189604,125189683,-,125189653,125189654,G,T,.,"AC=(1,);AF=(4.744000034406781e-05,);AN=18652;D...","['ENST00000373547.9', 'ENST00000415905.5', 'EN...",3,CCCCACAGCCGGAAGGGGCGAGCCCGCAAATAGGGCTCACCTTCAG...,CCCCACAGCCGGAAGGGGCGAGCCCGCAAATAGGGCTCACCTTCAG...,0.565911,0.437478,-0.072682,0.745286
24362,chr9,125189643,D,"[NM_001123369.2, NM_002721.5]",-,PPP6C,chr9,125189604,125189683,-,125189678,125189679,C,T,.,"AC=(1,);AF=(4.742999954032712e-05,);AN=18716;D...","['ENST00000373547.9', 'ENST00000415905.5', 'EN...",3,CCCCACAGCCGGAAGGGGCGAGCCCGCAAATAGGGCTCACCTTCAG...,CCCCACAGCCGGAAGGGGCGAGCCCGCAAATAGGGCTCACCTTCAG...,0.565911,0.467558,-0.055659,0.570064
24645,chr9,133345171,D,"[NM_133640.5, NM_181491.3]",-,MED22,chr9,133345150,133345229,-,133345181,133345182,G,A,rs199628533,"AC=(1,);AF=(2.370999936829321e-05,);AN=18684;D...",['ENST00000371999.5'],1,GCCCTCCACCCTGGCCACTCACGATGTTGGCGGCTCGCACATGCAT...,GCCCTCCACCCTGGCCACTCACGATGTTGGCAGCTCGCACATGCAT...,0.531890,0.372941,-0.084544,0.933939
24646,chr9,133345171,D,"[NM_133640.5, NM_181491.3]",-,MED22,chr9,133345150,133345229,-,133345186,133345187,G,A,.,"AC=(1,);AF=(7.113000174285844e-05,);AN=18642;D...",['ENST00000371999.5'],1,GCCCTCCACCCTGGCCACTCACGATGTTGGCGGCTCGCACATGCAT...,GCCCTCCACCCTGGCCACTCACGATGTTGGCGGCTCACACATGCAT...,0.531890,0.313002,-0.116425,1.318417


In [11]:
result['RefSeq_Transcript_ID'] = result['RefSeq_Transcript_ID'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
result['Transcript_id'] = result['Transcript_id'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
result

/tmp/ipykernel_1001848/2121069923.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['RefSeq_Transcript_ID'] = result['RefSeq_Transcript_ID'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
/tmp/ipykernel_1001848/2121069923.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Transcript_id'] = result['Transcript_id'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)


,chr,position,splice_type,RefSeq_Transcript_ID,strand,gene,chr_name,Donor_start,Donor_end,Strand,Variant_start,Variant_end,Reference_Nucleotide,Alternative_Nucleotides,DBSNP_ID,Info,Transcript_id,number_of_transcripts,Reference_Genome_Sequence,Alternative_Genome_Sequence,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
96,chr1,970758,D,"NM_001160184.2, NM_001367552.1, NM_032129.3",+,PLEKHN1,chr1,970720,970799,+,970720,970721,C,T,rs144236796,"AC=(5,);AF=(0.0003319999959785491,);AN=16852;D...","['ENST00000379407.7', 'ENST00000379410.8']",2,CGCTCGAGGGGTCCCGAGAGCACGCCTTCCAGATCACAGGTGTTTG...,TGCTCGAGGGGTCCCGAGAGCACGCCTTCCAGATCACAGGTGTTTG...,0.908698,0.203737,-0.640597,5.281620
97,chr1,970758,D,"NM_001160184.2, NM_001367552.1, NM_032129.3",+,PLEKHN1,chr1,970720,970799,+,970767,970768,G,T,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18790;D...","['ENST00000379407.7', 'ENST00000379410.8']",2,CGCTCGAGGGGTCCCGAGAGCACGCCTTCCAGATCACAGGTGTTTG...,CGCTCGAGGGGTCCCGAGAGCACGCCTTCCAGATCACAGGTGTTTG...,0.908698,0.214611,-0.630715,5.186764
338,chr1,974364,D,"NM_001160184.2, NM_001367552.1, NM_032129.3",+,PLEKHN1,chr1,974326,974405,+,974343,974344,G,A,.,"AC=(4,);AF=(0.00026080000679939985,);AN=18888;...","['ENST00000379407.7', 'ENST00000379409.6', 'EN...",4,CCAGGGAAGGTTCGCCCGAACCCTGGCTGCCTCTGACAGGTGAGTA...,CCAGGGAAGGTTCGCCCAAACCCTGGCTGCCTCTGACAGGTGAGTA...,0.716847,0.482730,-0.167826,1.439783
1203,chr1,11794728,D,"NM_001330358.2, NM_005957.5",-,MTHFR,chr1,11794689,11794768,-,11794733,11794734,G,A,rs200138092,"AC=(8,);AF=(0.00030819998937658966,);AN=18912;...","['ENST00000376583.7', 'ENST00000376585.6', 'EN...",11,AACAAAGATGTATTTGCAAGGAAGGTCTGCAGGCCCTCACCAGCGG...,AACAAAGATGTATTTGCAAGGAAGGTCTGCAGGCCCTCACCAGCAG...,0.648849,0.084613,-0.366104,4.321229
2956,chr1,44804779,D,NM_004073.4,+,PLK3,chr1,44804741,44804820,+,44804749,44804750,C,T,.,"AC=(1,);AF=(7.111999730113894e-05,);AN=18904;D...","['ENST00000372201.5', 'ENST00000465443.5']",2,CGGTACTTCGCCTCCTACATGGAGCAGCACCTCATGAAGGTGTGAG...,CGGTACTTTGCCTCCTACATGGAGCAGCACCTCATGAAGGTGTGAG...,0.732976,0.410285,-0.236525,1.980187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24361,chr9,125189643,D,"NM_001123369.2, NM_002721.5",-,PPP6C,chr9,125189604,125189683,-,125189653,125189654,G,T,.,"AC=(1,);AF=(4.744000034406781e-05,);AN=18652;D...","['ENST00000373547.9', 'ENST00000415905.5', 'EN...",3,CCCCACAGCCGGAAGGGGCGAGCCCGCAAATAGGGCTCACCTTCAG...,CCCCACAGCCGGAAGGGGCGAGCCCGCAAATAGGGCTCACCTTCAG...,0.565911,0.437478,-0.072682,0.745286
24362,chr9,125189643,D,"NM_001123369.2, NM_002721.5",-,PPP6C,chr9,125189604,125189683,-,125189678,125189679,C,T,.,"AC=(1,);AF=(4.742999954032712e-05,);AN=18716;D...","['ENST00000373547.9', 'ENST00000415905.5', 'EN...",3,CCCCACAGCCGGAAGGGGCGAGCCCGCAAATAGGGCTCACCTTCAG...,CCCCACAGCCGGAAGGGGCGAGCCCGCAAATAGGGCTCACCTTCAG...,0.565911,0.467558,-0.055659,0.570064
24645,chr9,133345171,D,"NM_133640.5, NM_181491.3",-,MED22,chr9,133345150,133345229,-,133345181,133345182,G,A,rs199628533,"AC=(1,);AF=(2.370999936829321e-05,);AN=18684;D...",['ENST00000371999.5'],1,GCCCTCCACCCTGGCCACTCACGATGTTGGCGGCTCGCACATGCAT...,GCCCTCCACCCTGGCCACTCACGATGTTGGCAGCTCGCACATGCAT...,0.531890,0.372941,-0.084544,0.933939
24646,chr9,133345171,D,"NM_133640.5, NM_181491.3",-,MED22,chr9,133345150,133345229,-,133345186,133345187,G,A,.,"AC=(1,);AF=(7.113000174285844e-05,);AN=18642;D...",['ENST00000371999.5'],1,GCCCTCCACCCTGGCCACTCACGATGTTGGCGGCTCGCACATGCAT...,GCCCTCCACCCTGGCCACTCACGATGTTGGCGGCTCACACATGCAT...,0.531890,0.313002,-0.116425,1.318417


In [12]:
# Unique instances from df_splice_donor that matched with df_donor_candidate
unique_splice_donor_from_data = result.drop_duplicates(subset=['chr', 'position', 'splice_type', 'RefSeq_Transcript_ID', 'strand', 'gene'])
unique_splice_donor_from_data

,chr,position,splice_type,RefSeq_Transcript_ID,strand,gene,chr_name,Donor_start,Donor_end,Strand,Variant_start,Variant_end,Reference_Nucleotide,Alternative_Nucleotides,DBSNP_ID,Info,Transcript_id,number_of_transcripts,Reference_Genome_Sequence,Alternative_Genome_Sequence,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
96,chr1,970758,D,"NM_001160184.2, NM_001367552.1, NM_032129.3",+,PLEKHN1,chr1,970720,970799,+,970720,970721,C,T,rs144236796,"AC=(5,);AF=(0.0003319999959785491,);AN=16852;D...","['ENST00000379407.7', 'ENST00000379410.8']",2,CGCTCGAGGGGTCCCGAGAGCACGCCTTCCAGATCACAGGTGTTTG...,TGCTCGAGGGGTCCCGAGAGCACGCCTTCCAGATCACAGGTGTTTG...,0.908698,0.203737,-0.640597,5.281620
338,chr1,974364,D,"NM_001160184.2, NM_001367552.1, NM_032129.3",+,PLEKHN1,chr1,974326,974405,+,974343,974344,G,A,.,"AC=(4,);AF=(0.00026080000679939985,);AN=18888;...","['ENST00000379407.7', 'ENST00000379409.6', 'EN...",4,CCAGGGAAGGTTCGCCCGAACCCTGGCTGCCTCTGACAGGTGAGTA...,CCAGGGAAGGTTCGCCCAAACCCTGGCTGCCTCTGACAGGTGAGTA...,0.716847,0.482730,-0.167826,1.439783
1203,chr1,11794728,D,"NM_001330358.2, NM_005957.5",-,MTHFR,chr1,11794689,11794768,-,11794733,11794734,G,A,rs200138092,"AC=(8,);AF=(0.00030819998937658966,);AN=18912;...","['ENST00000376583.7', 'ENST00000376585.6', 'EN...",11,AACAAAGATGTATTTGCAAGGAAGGTCTGCAGGCCCTCACCAGCGG...,AACAAAGATGTATTTGCAAGGAAGGTCTGCAGGCCCTCACCAGCAG...,0.648849,0.084613,-0.366104,4.321229
2956,chr1,44804779,D,NM_004073.4,+,PLK3,chr1,44804741,44804820,+,44804749,44804750,C,T,.,"AC=(1,);AF=(7.111999730113894e-05,);AN=18904;D...","['ENST00000372201.5', 'ENST00000465443.5']",2,CGGTACTTCGCCTCCTACATGGAGCAGCACCTCATGAAGGTGTGAG...,CGGTACTTTGCCTCCTACATGGAGCAGCACCTCATGAAGGTGTGAG...,0.732976,0.410285,-0.236525,1.980187
3173,chr1,52854630,D,"NM_001004339.3, NM_001307931.2",+,ZYG11A,chr1,52854592,52854671,+,52854603,52854604,C,T,.,"AC=(4,);AF=(9.483999747317284e-05,);AN=18868;D...","['ENST00000371528.2', 'ENST00000371532.5', 'EN...",3,AAGTAGCCGAGCGATTTCTCAGGGTGATGACTTGGCAAGGTAGTGA...,AAGTAGCCGAGTGATTTCTCAGGGTGATGACTTGGCAAGGTAGTGA...,0.758159,0.418518,-0.257502,2.122888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24007,chr8,43103830,D,"NM_001277971.2, NM_032237.5",+,POMK,chr8,43103792,43103871,+,43103804,43103805,C,T,rs370978230,"AC=(2,);AF=(4.741999873658642e-05,);AN=18846;D...","['ENST00000331373.10', 'ENST00000614426.2', 'E...",5,AGACAGCTGAAGCGTGTTGGGGAAGGAGCTGTAAAGAGAGTGAGTC...,AGACAGCTGAAGTGTGTTGGGGAAGGAGCTGTAAAGAGAGTGAGTC...,0.596648,0.286432,-0.185090,1.881697
24246,chr9,69065035,D,NM_000144.5,+,FXN,chr9,69064997,69065076,+,69065036,69065037,G,C,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18872;D...","['ENST00000377270.8', 'ENST00000484259.3', 'EN...",8,GACGCCAAACAAGCAAATCTGGCTATCTTCTCCATCCAGGTATGTA...,GACGCCAAACAAGCAAATCTGGCTATCTTCTCCATCCAGCTATGTA...,0.996860,0.358269,-0.636585,9.151310
24252,chr9,69065043,D,NM_181425.3,+,FXN,chr9,69064997,69065076,+,69065036,69065037,G,C,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18872;D...","['ENST00000377270.8', 'ENST00000484259.3', 'EN...",8,GACGCCAAACAAGCAAATCTGGCTATCTTCTCCATCCAGGTATGTA...,GACGCCAAACAAGCAAATCTGGCTATCTTCTCCATCCAGCTATGTA...,0.996860,0.358269,-0.636585,9.151310
24361,chr9,125189643,D,"NM_001123369.2, NM_002721.5",-,PPP6C,chr9,125189604,125189683,-,125189653,125189654,G,T,.,"AC=(1,);AF=(4.744000034406781e-05,);AN=18652;D...","['ENST00000373547.9', 'ENST00000415905.5', 'EN...",3,CCCCACAGCCGGAAGGGGCGAGCCCGCAAATAGGGCTCACCTTCAG...,CCCCACAGCCGGAAGGGGCGAGCCCGCAAATAGGGCTCACCTTCAG...,0.565911,0.437478,-0.072682,0.745286


In [13]:
unique_candidate_splice_donor = result.drop_duplicates(subset=['chr_name','Donor_start', 'Donor_end', 'Strand',	'Variant_start', 'Variant_end',	'Reference_Nucleotide',	'Alternative_Nucleotides', 'DBSNP_ID'])
unique_candidate_splice_donor

,chr,position,splice_type,RefSeq_Transcript_ID,strand,gene,chr_name,Donor_start,Donor_end,Strand,Variant_start,Variant_end,Reference_Nucleotide,Alternative_Nucleotides,DBSNP_ID,Info,Transcript_id,number_of_transcripts,Reference_Genome_Sequence,Alternative_Genome_Sequence,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
96,chr1,970758,D,"NM_001160184.2, NM_001367552.1, NM_032129.3",+,PLEKHN1,chr1,970720,970799,+,970720,970721,C,T,rs144236796,"AC=(5,);AF=(0.0003319999959785491,);AN=16852;D...","['ENST00000379407.7', 'ENST00000379410.8']",2,CGCTCGAGGGGTCCCGAGAGCACGCCTTCCAGATCACAGGTGTTTG...,TGCTCGAGGGGTCCCGAGAGCACGCCTTCCAGATCACAGGTGTTTG...,0.908698,0.203737,-0.640597,5.281620
97,chr1,970758,D,"NM_001160184.2, NM_001367552.1, NM_032129.3",+,PLEKHN1,chr1,970720,970799,+,970767,970768,G,T,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18790;D...","['ENST00000379407.7', 'ENST00000379410.8']",2,CGCTCGAGGGGTCCCGAGAGCACGCCTTCCAGATCACAGGTGTTTG...,CGCTCGAGGGGTCCCGAGAGCACGCCTTCCAGATCACAGGTGTTTG...,0.908698,0.214611,-0.630715,5.186764
338,chr1,974364,D,"NM_001160184.2, NM_001367552.1, NM_032129.3",+,PLEKHN1,chr1,974326,974405,+,974343,974344,G,A,.,"AC=(4,);AF=(0.00026080000679939985,);AN=18888;...","['ENST00000379407.7', 'ENST00000379409.6', 'EN...",4,CCAGGGAAGGTTCGCCCGAACCCTGGCTGCCTCTGACAGGTGAGTA...,CCAGGGAAGGTTCGCCCAAACCCTGGCTGCCTCTGACAGGTGAGTA...,0.716847,0.482730,-0.167826,1.439783
1203,chr1,11794728,D,"NM_001330358.2, NM_005957.5",-,MTHFR,chr1,11794689,11794768,-,11794733,11794734,G,A,rs200138092,"AC=(8,);AF=(0.00030819998937658966,);AN=18912;...","['ENST00000376583.7', 'ENST00000376585.6', 'EN...",11,AACAAAGATGTATTTGCAAGGAAGGTCTGCAGGCCCTCACCAGCGG...,AACAAAGATGTATTTGCAAGGAAGGTCTGCAGGCCCTCACCAGCAG...,0.648849,0.084613,-0.366104,4.321229
2956,chr1,44804779,D,NM_004073.4,+,PLK3,chr1,44804741,44804820,+,44804749,44804750,C,T,.,"AC=(1,);AF=(7.111999730113894e-05,);AN=18904;D...","['ENST00000372201.5', 'ENST00000465443.5']",2,CGGTACTTCGCCTCCTACATGGAGCAGCACCTCATGAAGGTGTGAG...,CGGTACTTTGCCTCCTACATGGAGCAGCACCTCATGAAGGTGTGAG...,0.732976,0.410285,-0.236525,1.980187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24361,chr9,125189643,D,"NM_001123369.2, NM_002721.5",-,PPP6C,chr9,125189604,125189683,-,125189653,125189654,G,T,.,"AC=(1,);AF=(4.744000034406781e-05,);AN=18652;D...","['ENST00000373547.9', 'ENST00000415905.5', 'EN...",3,CCCCACAGCCGGAAGGGGCGAGCCCGCAAATAGGGCTCACCTTCAG...,CCCCACAGCCGGAAGGGGCGAGCCCGCAAATAGGGCTCACCTTCAG...,0.565911,0.437478,-0.072682,0.745286
24362,chr9,125189643,D,"NM_001123369.2, NM_002721.5",-,PPP6C,chr9,125189604,125189683,-,125189678,125189679,C,T,.,"AC=(1,);AF=(4.742999954032712e-05,);AN=18716;D...","['ENST00000373547.9', 'ENST00000415905.5', 'EN...",3,CCCCACAGCCGGAAGGGGCGAGCCCGCAAATAGGGCTCACCTTCAG...,CCCCACAGCCGGAAGGGGCGAGCCCGCAAATAGGGCTCACCTTCAG...,0.565911,0.467558,-0.055659,0.570064
24645,chr9,133345171,D,"NM_133640.5, NM_181491.3",-,MED22,chr9,133345150,133345229,-,133345181,133345182,G,A,rs199628533,"AC=(1,);AF=(2.370999936829321e-05,);AN=18684;D...",['ENST00000371999.5'],1,GCCCTCCACCCTGGCCACTCACGATGTTGGCGGCTCGCACATGCAT...,GCCCTCCACCCTGGCCACTCACGATGTTGGCAGCTCGCACATGCAT...,0.531890,0.372941,-0.084544,0.933939
24646,chr9,133345171,D,"NM_133640.5, NM_181491.3",-,MED22,chr9,133345150,133345229,-,133345186,133345187,G,A,.,"AC=(1,);AF=(7.113000174285844e-05,);AN=18642;D...",['ENST00000371999.5'],1,GCCCTCCACCCTGGCCACTCACGATGTTGGCGGCTCGCACATGCAT...,GCCCTCCACCCTGGCCACTCACGATGTTGGCGGCTCACACATGCAT...,0.531890,0.313002,-0.116425,1.318417


In [13]:
# Finding the duplicate rows based on the specified subset of columns
duplicates = result.duplicated(subset=['chr_name','Donor_start', 'Donor_end', 'Strand', 'Variant_start', 'Variant_end', 'Reference_Nucleotide', 'Alternative_Nucleotides', 'DBSNP_ID'], keep=False)

# Displaying only the duplicate rows from the result DataFrame
duplicate_rows = result[duplicates]
duplicate_rows

,chr,position,splice_type,RefSeq_Transcript_ID,strand,gene,chr_name,Donor_start,Donor_end,Strand,Variant_start,Variant_end,Reference_Nucleotide,Alternative_Nucleotides,DBSNP_ID,Info,Transcript_id,number_of_transcripts,Reference_Genome_Sequence,Alternative_Genome_Sequence,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
9381,chr1,178513227,D,"NM_001170723.2, NM_001170724.2, NM_032126.5",+,TEX35,chr1,178513189,178513268,+,178513228,178513229,G,T,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18914;D...","['ENST00000319416.7', 'ENST00000367641.7', 'EN...",5,ATGTCGGCCAAGAGGGCAGAATTGAAGAAAACACATCTGGTAAAAG...,ATGTCGGCCAAGAGGGCAGAATTGAAGAAAACACATCTGTTAAAAG...,0.938361,0.129994,-0.758540,6.670807
9382,chr1,178513227,D,"NM_001170723.2, NM_001170724.2, NM_032126.5",+,TEX35,chr1,178513189,178513268,+,178513231,178513232,A,T,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18912;D...","['ENST00000319416.7', 'ENST00000367641.7', 'EN...",5,ATGTCGGCCAAGAGGGCAGAATTGAAGAAAACACATCTGGTAAAAG...,ATGTCGGCCAAGAGGGCAGAATTGAAGAAAACACATCTGGTATAAG...,0.938361,0.396214,-0.508730,4.535989
9405,chr1,178513251,D,NM_001170722.2,+,TEX35,chr1,178513189,178513268,+,178513228,178513229,G,T,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18914;D...","['ENST00000319416.7', 'ENST00000367641.7', 'EN...",5,ATGTCGGCCAAGAGGGCAGAATTGAAGAAAACACATCTGGTAAAAG...,ATGTCGGCCAAGAGGGCAGAATTGAAGAAAACACATCTGTTAAAAG...,0.938361,0.129994,-0.758540,6.670807
9406,chr1,178513251,D,NM_001170722.2,+,TEX35,chr1,178513189,178513268,+,178513231,178513232,A,T,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18912;D...","['ENST00000319416.7', 'ENST00000367641.7', 'EN...",5,ATGTCGGCCAAGAGGGCAGAATTGAAGAAAACACATCTGGTAAAAG...,ATGTCGGCCAAGAGGGCAGAATTGAAGAAAACACATCTGGTATAAG...,0.938361,0.396214,-0.508730,4.535989
20185,chr3,38140568,D,"NM_001172568.2, NM_001374787.1, NM_002468.5",+,MYD88,chr3,38140554,38140633,+,38140585,38140587,CG,C,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18828;D...",['ENST00000421516.3'],1,GCTCATCGAAAAGAGGTTGGCTAGAAGGCCACGGGGTGGGTGCGTG...,GCTCATCGAAAAGAGGTTGGCTAGAAGGCCACGGGTGGGTGCGTGG...,0.984457,0.117391,-0.853590,8.895473
20192,chr3,38140592,D,NM_001172567.2,+,MYD88,chr3,38140554,38140633,+,38140585,38140587,CG,C,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18828;D...",['ENST00000421516.3'],1,GCTCATCGAAAAGAGGTTGGCTAGAAGGCCACGGGGTGGGTGCGTG...,GCTCATCGAAAAGAGGTTGGCTAGAAGGCCACGGGTGGGTGCGTGG...,0.984457,0.117391,-0.853590,8.895473
24246,chr9,69065035,D,NM_000144.5,+,FXN,chr9,69064997,69065076,+,69065036,69065037,G,C,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18872;D...","['ENST00000377270.8', 'ENST00000484259.3', 'EN...",8,GACGCCAAACAAGCAAATCTGGCTATCTTCTCCATCCAGGTATGTA...,GACGCCAAACAAGCAAATCTGGCTATCTTCTCCATCCAGCTATGTA...,0.996860,0.358269,-0.636585,9.151310
24252,chr9,69065043,D,NM_181425.3,+,FXN,chr9,69064997,69065076,+,69065036,69065037,G,C,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18872;D...","['ENST00000377270.8', 'ENST00000484259.3', 'EN...",8,GACGCCAAACAAGCAAATCTGGCTATCTTCTCCATCCAGGTATGTA...,GACGCCAAACAAGCAAATCTGGCTATCTTCTCCATCCAGCTATGTA...,0.996860,0.358269,-0.636585,9.151310


In [14]:
df_splice_acceptor= unique_splice_df[unique_splice_df['splice_type']=="A"]
df_splice_acceptor

,chr,position,splice_type,RefSeq_Transcript_ID,strand,gene
1,chr1,966703,A,"[NM_001160184.2, NM_001367552.1, NM_032129.3]",+,PLEKHN1
3,chr1,970276,A,"[NM_001160184.2, NM_001367552.1, NM_032129.3]",+,PLEKHN1
5,chr1,970520,A,"[NM_001160184.2, NM_001367552.1, NM_032129.3]",+,PLEKHN1
7,chr1,970685,A,"[NM_001160184.2, NM_001367552.1, NM_032129.3]",+,PLEKHN1
9,chr1,970878,A,"[NM_001160184.2, NM_001367552.1, NM_032129.3]",+,PLEKHN1
...,...,...,...,...,...,...
5817,chrX,151699995,A,"[NM_001372163.1, NM_024082.4]",+,PRRG3
5818,chrX,151700006,A,NM_001372165.1,+,PRRG3
5820,chrX,151700505,A,"[NM_001372163.1, NM_001372165.1, NM_024082.4]",+,PRRG3
5821,chrX,154478058,A,NM_006014.5,-,LAGE3


In [15]:
df_acceptor_candidate = pd.read_csv("/data/private/pdutta/Collab_data/Oliver_data/New_data/Output_data/Acceptor/candiadte_variants.tsv", sep="\t")
df_acceptor_candidate

,chr_name,Acceptor_start,Acceptor_end,Strand,Variant_start,Variant_end,Reference_Nucleotide,Alternative_Nucleotides,DBSNP_ID,Info,Transcript_id,number_of_transcripts,Reference_Genome_Sequence,Alternative_Genome_Sequence,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
0,chr1,1478603,1478682,+,1478677,1478678,G,A,.,"AC=(1,);AF=(4.754000110551715e-05,);AN=18854;D...",['ENST00000673477.1'],1,GCCCTGAGCAAGTGCCAGCTGGTGAGTGCTGTGCTCTGCAGGAGTA...,GCCCTGAGCAAGTGCCAGCTGGTGAGTGCTGTGCTCTGCAGGAGTA...,0.916452,0.266757,-0.595414,4.914142
1,chr1,11803090,11803169,-,11803096,11803098,TC,T,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18888;D...","['ENST00000376583.7', 'ENST00000376585.6', 'EN...",12,GCTGTTTCCTCTGGCTTCGTTCACCATGGCTGGGTTCCTACTGCAC...,GCTGTTTCTCTGGCTTCGTTCACCATGGCTGGGTTCCTACTGCACA...,0.986711,0.039305,-0.934816,10.825621
2,chr1,44800995,44801074,+,44801036,44801037,A,T,.,"AC=(2,);AF=(4.741999873658642e-05,);AN=18914;D...","['ENST00000372201.5', 'ENST00000465443.5']",2,GGGAACCATGGAAGGATGACGACTCCGCGCCCTCATCGCAGATCCT...,GGGAACCATGGAAGGATGACGACTCCGCGCCCTCATCGCAGTTCCT...,0.734151,0.272161,-0.339170,2.884627
3,chr1,178516552,178516631,+,178516621,178516622,G,T,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18912;D...",['ENST00000419909.6'],1,GCCACAACTAACCTCAATAACATGCTCTGTGCTTTGTCAAGCCATG...,GCCACAACTAACCTCAATAACATGCTCTGTGCTTTGTCAAGCCATG...,0.979957,0.426352,-0.542509,6.039687
4,chr1,178520331,178520410,+,178520391,178520392,A,G,.,"AC=(1,);AF=(4.7410001570824534e-05,);AN=18902;...","['ENST00000319416.7', 'ENST00000367639.1', 'EN...",5,CCTTCAAAATGAAGATGCTAGTTCTGAATTCTCTCTCGAAGGAAAT...,CCTTCAAAATGAAGATGCTAGTTCTGAATTCTCTCTCGAAGGAAAT...,0.994082,0.414818,-0.575836,7.888524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,chr9,127817164,127817243,-,127817178,127817179,C,T,rs138799379,"AC=(2,);AF=(7.113999890862033e-05,);AN=18888;D...","['ENST00000344849.4', 'ENST00000373203.9', 'EN...",3,TGATGATGTTCAAGCGCATGAAGACAGTCCTATGGACTTCCTGGAG...,TGATGATGTTCAAGTGCATGAAGACAGTCCTATGGACTTCCTGGAG...,0.580133,0.243287,-0.195415,2.103537
60,chr9,129068157,129068236,+,129068227,129068228,G,T,rs201535391,"AC=(4,);AF=(0.0001897000038297847,);AN=18588;D...","['ENST00000358369.8', 'ENST00000439290.5', 'EN...",5,AGTGCCCCCATGCATGAGCCTCCCGGGGGCACCCTCTGTAGGTGGT...,AGTGCCCCCATGCATGAGCCTCCCGGGGGCACCCTCTGTAGGTGGT...,0.919668,0.231644,-0.632755,5.246943
61,chr9,129810186,129810265,+,129810226,129810227,G,T,.,"AC=(1,);AF=(2.4130000383593142e-05,);AN=18908;...","['ENST00000427860.1', 'ENST00000488169.1']",2,GGCCAAAAACATCCTTTTGCTCTGTCTCGTTCTTTACACAGAGTTC...,GGCCAAAAACATCCTTTTGCTCTGTCTCGTTCTTTACACATAGTTC...,0.999912,0.428342,-0.571520,13.894660
62,chr9,136211413,136211492,-,136211436,136211437,G,A,.,"AC=(1,);AF=(4.741999873658642e-05,);AN=18888;D...","['ENST00000358701.10', 'ENST00000455222.1']",2,CTCATTGTCTGCAGCACAGCCTGGGGGTCGTCTTCAAAGCCTGCAG...,CTCATTGTCTGCAGCACAGCCTGAGGGTCGTCTTCAAAGCCTGCAG...,0.991046,0.381109,-0.604476,7.489742


In [20]:
# Find rows of df_splice_donor where the position resides between Donor_start and Donor_end of df_donor_candidate
merged_df_acceptor = pd.merge(df_splice_acceptor, df_acceptor_candidate, left_on="chr", right_on="chr_name")
result = merged_df_acceptor[(merged_df_acceptor['position'] >= merged_df_acceptor['Acceptor_start']) & (merged_df_acceptor['position'] <= merged_df_acceptor['Acceptor_end'])]
result

,chr,position,splice_type,RefSeq_Transcript_ID,strand,gene,chr_name,Acceptor_start,Acceptor_end,Strand,Variant_start,Variant_end,Reference_Nucleotide,Alternative_Nucleotides,DBSNP_ID,Info,Transcript_id,number_of_transcripts,Reference_Genome_Sequence,Alternative_Genome_Sequence,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
231,chr1,1478643,A,NM_031921.6,+,ATAD3B,chr1,1478603,1478682,+,1478677,1478678,G,A,.,"AC=(1,);AF=(4.754000110551715e-05,);AN=18854;D...",['ENST00000673477.1'],1,GCCCTGAGCAAGTGCCAGCTGGTGAGTGCTGTGCTCTGCAGGAGTA...,GCCCTGAGCAAGTGCCAGCTGGTGAGTGCTGTGCTCTGCAGGAGTA...,0.916452,0.266757,-0.595414,4.914142
400,chr1,11803129,A,"[NM_001330358.2, NM_005957.5]",-,MTHFR,chr1,11803090,11803169,-,11803096,11803098,TC,T,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18888;D...","['ENST00000376583.7', 'ENST00000376585.6', 'EN...",12,GCTGTTTCCTCTGGCTTCGTTCACCATGGCTGGGTTCCTACTGCAC...,GCTGTTTCTCTGGCTTCGTTCACCATGGCTGGGTTCCTACTGCACA...,0.986711,0.039305,-0.934816,10.825621
702,chr1,44801035,A,NM_004073.4,+,PLK3,chr1,44800995,44801074,+,44801036,44801037,A,T,.,"AC=(2,);AF=(4.741999873658642e-05,);AN=18914;D...","['ENST00000372201.5', 'ENST00000465443.5']",2,GGGAACCATGGAAGGATGACGACTCCGCGCCCTCATCGCAGATCCT...,GGGAACCATGGAAGGATGACGACTCCGCGCCCTCATCGCAGTTCCT...,0.734151,0.272161,-0.339170,2.884627
2663,chr1,178516614,A,"[NM_001170722.2, NM_001170723.2, NM_001170724....",+,TEX35,chr1,178516552,178516631,+,178516621,178516622,G,T,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18912;D...",['ENST00000419909.6'],1,GCCACAACTAACCTCAATAACATGCTCTGTGCTTTGTCAAGCCATG...,GCCACAACTAACCTCAATAACATGCTCTGTGCTTTGTCAAGCCATG...,0.979957,0.426352,-0.542509,6.039687
2671,chr1,178520371,A,"[NM_001170722.2, NM_001170723.2, NM_001170724....",+,TEX35,chr1,178520331,178520410,+,178520391,178520392,A,G,.,"AC=(1,);AF=(4.7410001570824534e-05,);AN=18902;...","['ENST00000319416.7', 'ENST00000367639.1', 'EN...",5,CCTTCAAAATGAAGATGCTAGTTCTGAATTCTCTCTCGAAGGAAAT...,CCTTCAAAATGAAGATGCTAGTTCTGAATTCTCTCTCGAAGGAAAT...,0.994082,0.414818,-0.575836,7.888524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448,chr9,127817203,A,"[NM_000118.4, NM_001114753.3, NM_001278138.2]",-,ENG,chr9,127817164,127817243,-,127817178,127817179,C,T,rs138799379,"AC=(2,);AF=(7.113999890862033e-05,);AN=18888;D...","['ENST00000344849.4', 'ENST00000373203.9', 'EN...",3,TGATGATGTTCAAGCGCATGAAGACAGTCCTATGGACTTCCTGGAG...,TGATGATGTTCAAGTGCATGAAGACAGTCCTATGGACTTCCTGGAG...,0.580133,0.243287,-0.195415,2.103537
10564,chr9,129068197,A,"[NM_001329990.2, NM_032809.4]",+,MIGA2,chr9,129068157,129068236,+,129068227,129068228,G,T,rs201535391,"AC=(4,);AF=(0.0001897000038297847,);AN=18588;D...","['ENST00000358369.8', 'ENST00000439290.5', 'EN...",5,AGTGCCCCCATGCATGAGCCTCCCGGGGGCACCCTCTGTAGGTGGT...,AGTGCCCCCATGCATGAGCCTCCCGGGGGCACCCTCTGTAGGTGGT...,0.919668,0.231644,-0.632755,5.246943
10605,chr9,129810226,A,"[NM_001317893.2, NM_001317894.2]",+,TOR1B,chr9,129810186,129810265,+,129810226,129810227,G,T,.,"AC=(1,);AF=(2.4130000383593142e-05,);AN=18908;...","['ENST00000427860.1', 'ENST00000488169.1']",2,GGCCAAAAACATCCTTTTGCTCTGTCTCGTTCTTTACACAGAGTTC...,GGCCAAAAACATCCTTTTGCTCTGTCTCGTTCTTTACACATAGTTC...,0.999912,0.428342,-0.571520,13.894660
10686,chr9,136211452,A,NM_181701.4,-,QSOX2,chr9,136211413,136211492,-,136211436,136211437,G,A,.,"AC=(1,);AF=(4.741999873658642e-05,);AN=18888;D...","['ENST00000358701.10', 'ENST00000455222.1']",2,CTCATTGTCTGCAGCACAGCCTGGGGGTCGTCTTCAAAGCCTGCAG...,CTCATTGTCTGCAGCACAGCCTGAGGGTCGTCTTCAAAGCCTGCAG...,0.991046,0.381109,-0.604476,7.489742


In [21]:
result['RefSeq_Transcript_ID'] = result['RefSeq_Transcript_ID'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
result['Transcript_id'] = result['Transcript_id'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
result

/tmp/ipykernel_1001848/2121069923.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['RefSeq_Transcript_ID'] = result['RefSeq_Transcript_ID'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
/tmp/ipykernel_1001848/2121069923.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Transcript_id'] = result['Transcript_id'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)


,chr,position,splice_type,RefSeq_Transcript_ID,strand,gene,chr_name,Acceptor_start,Acceptor_end,Strand,Variant_start,Variant_end,Reference_Nucleotide,Alternative_Nucleotides,DBSNP_ID,Info,Transcript_id,number_of_transcripts,Reference_Genome_Sequence,Alternative_Genome_Sequence,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
231,chr1,1478643,A,NM_031921.6,+,ATAD3B,chr1,1478603,1478682,+,1478677,1478678,G,A,.,"AC=(1,);AF=(4.754000110551715e-05,);AN=18854;D...",['ENST00000673477.1'],1,GCCCTGAGCAAGTGCCAGCTGGTGAGTGCTGTGCTCTGCAGGAGTA...,GCCCTGAGCAAGTGCCAGCTGGTGAGTGCTGTGCTCTGCAGGAGTA...,0.916452,0.266757,-0.595414,4.914142
400,chr1,11803129,A,"NM_001330358.2, NM_005957.5",-,MTHFR,chr1,11803090,11803169,-,11803096,11803098,TC,T,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18888;D...","['ENST00000376583.7', 'ENST00000376585.6', 'EN...",12,GCTGTTTCCTCTGGCTTCGTTCACCATGGCTGGGTTCCTACTGCAC...,GCTGTTTCTCTGGCTTCGTTCACCATGGCTGGGTTCCTACTGCACA...,0.986711,0.039305,-0.934816,10.825621
702,chr1,44801035,A,NM_004073.4,+,PLK3,chr1,44800995,44801074,+,44801036,44801037,A,T,.,"AC=(2,);AF=(4.741999873658642e-05,);AN=18914;D...","['ENST00000372201.5', 'ENST00000465443.5']",2,GGGAACCATGGAAGGATGACGACTCCGCGCCCTCATCGCAGATCCT...,GGGAACCATGGAAGGATGACGACTCCGCGCCCTCATCGCAGTTCCT...,0.734151,0.272161,-0.339170,2.884627
2663,chr1,178516614,A,"NM_001170722.2, NM_001170723.2, NM_001170724.2...",+,TEX35,chr1,178516552,178516631,+,178516621,178516622,G,T,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18912;D...",['ENST00000419909.6'],1,GCCACAACTAACCTCAATAACATGCTCTGTGCTTTGTCAAGCCATG...,GCCACAACTAACCTCAATAACATGCTCTGTGCTTTGTCAAGCCATG...,0.979957,0.426352,-0.542509,6.039687
2671,chr1,178520371,A,"NM_001170722.2, NM_001170723.2, NM_001170724.2...",+,TEX35,chr1,178520331,178520410,+,178520391,178520392,A,G,.,"AC=(1,);AF=(4.7410001570824534e-05,);AN=18902;...","['ENST00000319416.7', 'ENST00000367639.1', 'EN...",5,CCTTCAAAATGAAGATGCTAGTTCTGAATTCTCTCTCGAAGGAAAT...,CCTTCAAAATGAAGATGCTAGTTCTGAATTCTCTCTCGAAGGAAAT...,0.994082,0.414818,-0.575836,7.888524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448,chr9,127817203,A,"NM_000118.4, NM_001114753.3, NM_001278138.2",-,ENG,chr9,127817164,127817243,-,127817178,127817179,C,T,rs138799379,"AC=(2,);AF=(7.113999890862033e-05,);AN=18888;D...","['ENST00000344849.4', 'ENST00000373203.9', 'EN...",3,TGATGATGTTCAAGCGCATGAAGACAGTCCTATGGACTTCCTGGAG...,TGATGATGTTCAAGTGCATGAAGACAGTCCTATGGACTTCCTGGAG...,0.580133,0.243287,-0.195415,2.103537
10564,chr9,129068197,A,"NM_001329990.2, NM_032809.4",+,MIGA2,chr9,129068157,129068236,+,129068227,129068228,G,T,rs201535391,"AC=(4,);AF=(0.0001897000038297847,);AN=18588;D...","['ENST00000358369.8', 'ENST00000439290.5', 'EN...",5,AGTGCCCCCATGCATGAGCCTCCCGGGGGCACCCTCTGTAGGTGGT...,AGTGCCCCCATGCATGAGCCTCCCGGGGGCACCCTCTGTAGGTGGT...,0.919668,0.231644,-0.632755,5.246943
10605,chr9,129810226,A,"NM_001317893.2, NM_001317894.2",+,TOR1B,chr9,129810186,129810265,+,129810226,129810227,G,T,.,"AC=(1,);AF=(2.4130000383593142e-05,);AN=18908;...","['ENST00000427860.1', 'ENST00000488169.1']",2,GGCCAAAAACATCCTTTTGCTCTGTCTCGTTCTTTACACAGAGTTC...,GGCCAAAAACATCCTTTTGCTCTGTCTCGTTCTTTACACATAGTTC...,0.999912,0.428342,-0.571520,13.894660
10686,chr9,136211452,A,NM_181701.4,-,QSOX2,chr9,136211413,136211492,-,136211436,136211437,G,A,.,"AC=(1,);AF=(4.741999873658642e-05,);AN=18888;D...","['ENST00000358701.10', 'ENST00000455222.1']",2,CTCATTGTCTGCAGCACAGCCTGGGGGTCGTCTTCAAAGCCTGCAG...,CTCATTGTCTGCAGCACAGCCTGAGGGTCGTCTTCAAAGCCTGCAG...,0.991046,0.381109,-0.604476,7.489742


In [22]:
unique_splice_acceptor_from_data = result.drop_duplicates(subset=['chr', 'position', 'splice_type', 'RefSeq_Transcript_ID', 'strand', 'gene'])
unique_splice_acceptor_from_data

,chr,position,splice_type,RefSeq_Transcript_ID,strand,gene,chr_name,Acceptor_start,Acceptor_end,Strand,Variant_start,Variant_end,Reference_Nucleotide,Alternative_Nucleotides,DBSNP_ID,Info,Transcript_id,number_of_transcripts,Reference_Genome_Sequence,Alternative_Genome_Sequence,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
231,chr1,1478643,A,NM_031921.6,+,ATAD3B,chr1,1478603,1478682,+,1478677,1478678,G,A,.,"AC=(1,);AF=(4.754000110551715e-05,);AN=18854;D...",['ENST00000673477.1'],1,GCCCTGAGCAAGTGCCAGCTGGTGAGTGCTGTGCTCTGCAGGAGTA...,GCCCTGAGCAAGTGCCAGCTGGTGAGTGCTGTGCTCTGCAGGAGTA...,0.916452,0.266757,-0.595414,4.914142
400,chr1,11803129,A,"NM_001330358.2, NM_005957.5",-,MTHFR,chr1,11803090,11803169,-,11803096,11803098,TC,T,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18888;D...","['ENST00000376583.7', 'ENST00000376585.6', 'EN...",12,GCTGTTTCCTCTGGCTTCGTTCACCATGGCTGGGTTCCTACTGCAC...,GCTGTTTCTCTGGCTTCGTTCACCATGGCTGGGTTCCTACTGCACA...,0.986711,0.039305,-0.934816,10.825621
702,chr1,44801035,A,NM_004073.4,+,PLK3,chr1,44800995,44801074,+,44801036,44801037,A,T,.,"AC=(2,);AF=(4.741999873658642e-05,);AN=18914;D...","['ENST00000372201.5', 'ENST00000465443.5']",2,GGGAACCATGGAAGGATGACGACTCCGCGCCCTCATCGCAGATCCT...,GGGAACCATGGAAGGATGACGACTCCGCGCCCTCATCGCAGTTCCT...,0.734151,0.272161,-0.339170,2.884627
2663,chr1,178516614,A,"NM_001170722.2, NM_001170723.2, NM_001170724.2...",+,TEX35,chr1,178516552,178516631,+,178516621,178516622,G,T,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18912;D...",['ENST00000419909.6'],1,GCCACAACTAACCTCAATAACATGCTCTGTGCTTTGTCAAGCCATG...,GCCACAACTAACCTCAATAACATGCTCTGTGCTTTGTCAAGCCATG...,0.979957,0.426352,-0.542509,6.039687
2671,chr1,178520371,A,"NM_001170722.2, NM_001170723.2, NM_001170724.2...",+,TEX35,chr1,178520331,178520410,+,178520391,178520392,A,G,.,"AC=(1,);AF=(4.7410001570824534e-05,);AN=18902;...","['ENST00000319416.7', 'ENST00000367639.1', 'EN...",5,CCTTCAAAATGAAGATGCTAGTTCTGAATTCTCTCTCGAAGGAAAT...,CCTTCAAAATGAAGATGCTAGTTCTGAATTCTCTCTCGAAGGAAAT...,0.994082,0.414818,-0.575836,7.888524
2770,chr1,206595838,A,"NM_001201478.2, NM_006893.3",-,EIF2D,chr1,206595799,206595878,-,206595809,206595810,T,A,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18914;D...","['ENST00000271764.7', 'ENST00000367114.7', 'EN...",7,AAGGGTCACTTGATAGGCAGGCTGTAATTTTTCCAAACACCTGAAA...,AAGGGTCACTAGATAGGCAGGCTGTAATTTTTCCAAACACCTGAAA...,0.553404,0.000119,-0.306190,13.350189
2848,chr1,206611374,A,"NM_001201478.2, NM_006893.3",-,EIF2D,chr1,206611335,206611414,-,206611362,206611363,T,A,.,"AC=(1,);AF=(2.3720000172033906e-05,);AN=18666;...","['ENST00000271764.7', 'ENST00000367114.7', 'EN...",5,GGTGGGGAAAGCAGTTGTCACATCAGCTCGAAGCTTTCTCCTGTGG...,GGTGGGGAAAGCAGTTGTCACATCAGCACGAAGCTTTCTCCTGTGG...,0.514838,0.405778,-0.056148,0.635963
3166,chr11,45903364,A,NM_005456.4,+,MAPK8IP1,chr11,45903324,45903403,+,45903402,45903403,C,T,rs72902473,"AC=(11,);AF=(0.0004031000135000795,);AN=18060;...","['ENST00000241014.6', 'ENST00000395629.2']",2,GAGCTGCGACCCCCCATCCAGCCACACCACCTCACCTGCAGGTGCT...,GAGCTGCGACCCCCCATCCAGCCACACCACCTCACCTGCAGGTGCT...,0.980594,0.251959,-0.714494,7.228986
3447,chr11,74374582,A,NM_173582.6,-,PGM2L1,chr11,74374543,74374622,-,74374575,74374576,T,G,.,"AC=(1,);AF=(2.3720000172033906e-05,);AN=18870;...",['ENST00000298198.5'],1,TCCGTAACAGGTTTTCAATCTGCTCTTTTGTTTTGGGATTCTATAA...,TCCGTAACAGGTTTTCAATCTGCTCTTTTGTTGTGGGATTCTATAA...,0.999408,0.119550,-0.879338,13.602491
3520,chr11,118658733,A,"NM_001301065.2, NM_007180.3",-,TREH,chr11,118658694,118658773,-,118658727,118658728,C,T,rs200479357,"AC=(2,);AF=(4.741999873658642e-05,);AN=18706;D...","['ENST00000264029.9', 'ENST00000397925.2', 'EN...",3,CTCCTCCCCCACCGGGCTGTCCACCGTTGCTGACGTCATACTGGGG...,CTCCTCCCCCACCGGGCTGTCCACCGTTGCTGATGTCATACTGGGG...,0.999237,0.407432,-0.591353,10.894561


In [24]:
unique_splice_acceptor_from_data.shape

(54, 24)

In [23]:
unique_candidate_splice_acceptor = result.drop_duplicates(subset=['chr_name','Acceptor_start', 'Acceptor_end', 'Strand',	'Variant_start', 'Variant_end',	'Reference_Nucleotide',	'Alternative_Nucleotides', 'DBSNP_ID'])
unique_candidate_splice_acceptor

,chr,position,splice_type,RefSeq_Transcript_ID,strand,gene,chr_name,Acceptor_start,Acceptor_end,Strand,Variant_start,Variant_end,Reference_Nucleotide,Alternative_Nucleotides,DBSNP_ID,Info,Transcript_id,number_of_transcripts,Reference_Genome_Sequence,Alternative_Genome_Sequence,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
231,chr1,1478643,A,NM_031921.6,+,ATAD3B,chr1,1478603,1478682,+,1478677,1478678,G,A,.,"AC=(1,);AF=(4.754000110551715e-05,);AN=18854;D...",['ENST00000673477.1'],1,GCCCTGAGCAAGTGCCAGCTGGTGAGTGCTGTGCTCTGCAGGAGTA...,GCCCTGAGCAAGTGCCAGCTGGTGAGTGCTGTGCTCTGCAGGAGTA...,0.916452,0.266757,-0.595414,4.914142
400,chr1,11803129,A,"NM_001330358.2, NM_005957.5",-,MTHFR,chr1,11803090,11803169,-,11803096,11803098,TC,T,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18888;D...","['ENST00000376583.7', 'ENST00000376585.6', 'EN...",12,GCTGTTTCCTCTGGCTTCGTTCACCATGGCTGGGTTCCTACTGCAC...,GCTGTTTCTCTGGCTTCGTTCACCATGGCTGGGTTCCTACTGCACA...,0.986711,0.039305,-0.934816,10.825621
702,chr1,44801035,A,NM_004073.4,+,PLK3,chr1,44800995,44801074,+,44801036,44801037,A,T,.,"AC=(2,);AF=(4.741999873658642e-05,);AN=18914;D...","['ENST00000372201.5', 'ENST00000465443.5']",2,GGGAACCATGGAAGGATGACGACTCCGCGCCCTCATCGCAGATCCT...,GGGAACCATGGAAGGATGACGACTCCGCGCCCTCATCGCAGTTCCT...,0.734151,0.272161,-0.339170,2.884627
2663,chr1,178516614,A,"NM_001170722.2, NM_001170723.2, NM_001170724.2...",+,TEX35,chr1,178516552,178516631,+,178516621,178516622,G,T,.,"AC=(1,);AF=(2.370999936829321e-05,);AN=18912;D...",['ENST00000419909.6'],1,GCCACAACTAACCTCAATAACATGCTCTGTGCTTTGTCAAGCCATG...,GCCACAACTAACCTCAATAACATGCTCTGTGCTTTGTCAAGCCATG...,0.979957,0.426352,-0.542509,6.039687
2671,chr1,178520371,A,"NM_001170722.2, NM_001170723.2, NM_001170724.2...",+,TEX35,chr1,178520331,178520410,+,178520391,178520392,A,G,.,"AC=(1,);AF=(4.7410001570824534e-05,);AN=18902;...","['ENST00000319416.7', 'ENST00000367639.1', 'EN...",5,CCTTCAAAATGAAGATGCTAGTTCTGAATTCTCTCTCGAAGGAAAT...,CCTTCAAAATGAAGATGCTAGTTCTGAATTCTCTCTCGAAGGAAAT...,0.994082,0.414818,-0.575836,7.888524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448,chr9,127817203,A,"NM_000118.4, NM_001114753.3, NM_001278138.2",-,ENG,chr9,127817164,127817243,-,127817178,127817179,C,T,rs138799379,"AC=(2,);AF=(7.113999890862033e-05,);AN=18888;D...","['ENST00000344849.4', 'ENST00000373203.9', 'EN...",3,TGATGATGTTCAAGCGCATGAAGACAGTCCTATGGACTTCCTGGAG...,TGATGATGTTCAAGTGCATGAAGACAGTCCTATGGACTTCCTGGAG...,0.580133,0.243287,-0.195415,2.103537
10564,chr9,129068197,A,"NM_001329990.2, NM_032809.4",+,MIGA2,chr9,129068157,129068236,+,129068227,129068228,G,T,rs201535391,"AC=(4,);AF=(0.0001897000038297847,);AN=18588;D...","['ENST00000358369.8', 'ENST00000439290.5', 'EN...",5,AGTGCCCCCATGCATGAGCCTCCCGGGGGCACCCTCTGTAGGTGGT...,AGTGCCCCCATGCATGAGCCTCCCGGGGGCACCCTCTGTAGGTGGT...,0.919668,0.231644,-0.632755,5.246943
10605,chr9,129810226,A,"NM_001317893.2, NM_001317894.2",+,TOR1B,chr9,129810186,129810265,+,129810226,129810227,G,T,.,"AC=(1,);AF=(2.4130000383593142e-05,);AN=18908;...","['ENST00000427860.1', 'ENST00000488169.1']",2,GGCCAAAAACATCCTTTTGCTCTGTCTCGTTCTTTACACAGAGTTC...,GGCCAAAAACATCCTTTTGCTCTGTCTCGTTCTTTACACATAGTTC...,0.999912,0.428342,-0.571520,13.894660
10686,chr9,136211452,A,NM_181701.4,-,QSOX2,chr9,136211413,136211492,-,136211436,136211437,G,A,.,"AC=(1,);AF=(4.741999873658642e-05,);AN=18888;D...","['ENST00000358701.10', 'ENST00000455222.1']",2,CTCATTGTCTGCAGCACAGCCTGGGGGTCGTCTTCAAAGCCTGCAG...,CTCATTGTCTGCAGCACAGCCTGAGGGTCGTCTTCAAAGCCTGCAG...,0.991046,0.381109,-0.604476,7.489742
